In [1]:
import folium
import pandas as pd
import geopy
import numpy as np
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [2]:
d2 = pd.read_csv('taxi+_zone_lookup.csv')
d1 = pd.read_csv('https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-02.csv')

In [3]:
d2.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [4]:
d2['Borough'].unique()

array(['EWR', 'Queens', 'Bronx', 'Manhattan', 'Staten Island', 'Brooklyn',
       'Unknown'], dtype=object)

In [5]:
#For this analysis, let's focus on just four top Boroughs - EWR, Queens, Brooklyn and Manhattan
d2 = d2.loc[d2['Borough'].isin(['EWR','Queens','Manhattan','Brooklyn'])]
d2['Borough'].unique()

array(['EWR', 'Queens', 'Manhattan', 'Brooklyn'], dtype=object)

In [6]:
#Let's check for the quality of the data before we merge the values
d2.isna().sum()

LocationID      0
Borough         0
Zone            0
service_zone    0
dtype: int64

In [7]:
#Now we don't have any null values, let's go ahead and concat join Zones and Boroughs to get a single location 
d2['location'] = d2['Zone'].str.cat(d2['Borough'], sep = " ")
d2['location'] = d2['location'] + " New York USA"

In [8]:
#Now let's geocode the locations to get the values of latitudes and longitudes
len(d2)

200

In [9]:
geolocator = Nominatim (user_agent = "taxidata")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds = 4)

d2['location_coordinate'] = d2['location'].apply(geocode)

d2['point'] = d2['location_coordinate'].apply(lambda loc: tuple(loc.point) if loc else None)

In [10]:
#d2.drop('lat',axis = 1, inplace = True)
d2.head()

,LocationID,Borough,Zone,service_zone,location,location_coordinate,point
0,1,EWR,Newark Airport,EWR,Newark Airport EWR New York USA,None,None
1,2,Queens,Jamaica Bay,Boro Zone,Jamaica Bay Queens New York USA,"(Jamaica Bay, Jamaica Bay Wildlife Refuge - We...","(40.6039936, -73.8354124, 0.0)"
3,4,Manhattan,Alphabet City,Yellow Zone,Alphabet City Manhattan New York USA,"(Alphabet City, Manhattan Community Board 3, N...","(40.7251022, -73.9795833, 0.0)"
6,7,Queens,Astoria,Boro Zone,Astoria Queens New York USA,"(Astoria–Ditmars Boulevard, 31st Street, Stein...","(40.7749872, -73.9121176, 0.0)"
7,8,Queens,Astoria Park,Boro Zone,Astoria Park Queens New York USA,"(Astoria Park, Queens, New York, United States...","(40.7788277, -73.92262648880973, 0.0)"


In [11]:
d2['lat'] = d2['point'].str[0]
d2['long'] = d2['point'].str[1]
d2_updated = d2.drop(['Borough','Zone','service_zone','location_coordinate','point'], axis = 1)
d2_updated

,LocationID,location,lat,long
0,1,Newark Airport EWR New York USA,NaN,NaN
1,2,Jamaica Bay Queens New York USA,40.603994,-73.835412
3,4,Alphabet City Manhattan New York USA,40.725102,-73.979583
6,7,Astoria Queens New York USA,40.774987,-73.912118
7,8,Astoria Park Queens New York USA,40.778828,-73.922626
...,...,...,...,...
257,258,Woodhaven Queens New York USA,40.689270,-73.857913
259,260,Woodside Queens New York USA,40.746160,-73.903285
260,261,World Trade Center Manhattan New York USA,40.711888,-74.012441
261,262,Yorkville East Manhattan New York USA,40.778007,-73.948202


In [12]:
zone_list = d2['LocationID'].tolist()

In [13]:
d1.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge'],
      dtype='object')

In [14]:
d1_updated = d1.loc[d1['PULocationID'].isin(zone_list) & d1['DOLocationID'].isin(zone_list)]

In [15]:
d1_final = d1_updated.drop(['RatecodeID','mta_tax','tip_amount','tolls_amount','extra','improvement_surcharge'
                ,'congestion_surcharge','store_and_fwd_flag','mta_tax'], axis = 1)

In [16]:
d1_final.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,payment_type,fare_amount,total_amount
0,1,2019-02-01 00:59:04,2019-02-01 01:07:27,1,2.1,48,234,1,9.0,12.3
1,1,2019-02-01 00:33:09,2019-02-01 01:03:58,1,9.8,230,93,2,32.0,33.3
2,1,2019-02-01 00:09:03,2019-02-01 00:09:16,1,0.0,145,145,2,2.5,3.8
3,1,2019-02-01 00:45:38,2019-02-01 00:51:10,1,0.8,95,95,2,5.5,6.8
4,1,2019-02-01 00:25:30,2019-02-01 00:28:14,1,0.8,140,263,2,5.0,6.3


In [17]:
d1_final['PU_lat'] = d1_final['PULocationID'].map(d2_updated.set_index('LocationID')['lat'])
d1_final['PU_long'] = d1_final['PULocationID'].map(d2_updated.set_index('LocationID')['long'])
d1_final['DO_lat'] = d1_final['DOLocationID'].map(d2_updated.set_index('LocationID')['lat'])
d1_final['DO_long'] = d1_final['DOLocationID'].map(d2_updated.set_index('LocationID')['long'])
d1_final.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,payment_type,fare_amount,total_amount,PU_lat,PU_long,DO_lat,DO_long
0,1,2019-02-01 00:59:04,2019-02-01 01:07:27,1,2.1,48,234,1,9.0,12.3,NaN,NaN,40.735909,-73.990163
1,1,2019-02-01 00:33:09,2019-02-01 01:03:58,1,9.8,230,93,2,32.0,33.3,NaN,NaN,40.740681,-73.843237
2,1,2019-02-01 00:09:03,2019-02-01 00:09:16,1,0.0,145,145,2,2.5,3.8,NaN,NaN,NaN,NaN
3,1,2019-02-01 00:45:38,2019-02-01 00:51:10,1,0.8,95,95,2,5.5,6.8,40.719594,-73.844855,40.719594,-73.844855
4,1,2019-02-01 00:25:30,2019-02-01 00:28:14,1,0.8,140,263,2,5.0,6.3,40.766437,-73.959017,40.778007,-73.948202


In [18]:
d1_final.dropna(how = 'any', inplace = True)

In [19]:
d1_final.isna().sum()

VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
PULocationID             0
DOLocationID             0
payment_type             0
fare_amount              0
total_amount             0
PU_lat                   0
PU_long                  0
DO_lat                   0
DO_long                  0
dtype: int64

In [20]:
# trial = d1_final.head(10)

# PU_locations = trial[['PU_lat','PU_long']].values.tolist()

# map1 = folium.Map(location=[40.7128, -74.0060], zoom_start = 10)

# for point in range(0,len(PU_locations)):
#     folium.Marker(PU_locations[point], popup = "Pickup_Point").add_to(map1)
# map1

In [24]:
d1_final.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,payment_type,fare_amount,total_amount,PU_lat,PU_long,DO_lat,DO_long
3,1,2019-02-01 00:45:38,2019-02-01 00:51:10,1,0.80,95,95,2,5.5,6.80,40.719594,-73.844855,40.719594,-73.844855
4,1,2019-02-01 00:25:30,2019-02-01 00:28:14,1,0.80,140,263,2,5.0,6.30,40.766437,-73.959017,40.778007,-73.948202
6,1,2019-02-01 00:06:49,2019-02-01 00:10:34,1,0.90,75,41,2,5.0,6.30,40.794722,-73.942500,40.807879,-73.945415
9,2,2019-01-31 23:16:28,2019-01-31 23:19:11,1,0.49,170,234,1,4.0,7.00,40.748157,-73.978750,40.735909,-73.990163
10,2,2019-01-31 23:28:02,2019-01-31 23:36:34,1,1.61,107,161,1,8.0,11.62,40.735519,-73.984079,40.760109,-73.978163


In [43]:
import folium.plugins as plugins


max_records = 3000

NY_coordinates = (40.7128, -74.0060)

map3 = folium.Map(location= NY_coordinates, zoom_start = 10 )


for i in d1_final[0:max_records].iterrows():
    folium.Marker(
    location = list([i[1]['PU_lat'],i[1]['PU_long']])).add_to(map3)

map3